##◢ Setup

In [0]:
#@title Setup DeOldify {display-mode: "form"}
#@markdown Click the play button on the left to run this block.
#@markdown
#@markdown This downloads the DeOldify repository and installs dependencies.
#@markdown
#@markdown Downloading the models can take some time.
#@markdown
#@markdown This block must be re-run if the runtime expires, usually because of inactivity.

#@markdown Tips:
#@markdown - Edit -> Clear all ouputs
#@markdown - Edit -> Show/hide code
#@markdown - Runtime -> Reset all runtimes
#@markdown - Runtime -> Manage sessions
!cd /content
!git clone https://github.com/jantic/DeOldify.git DeOldify 
%cd /content/DeOldify
!pip install -q -r requirements.txt
!apt install -qq zip

# Downloads "Artistic" model (artistic = true) (recommended)
!mkdir 'models'
!wget wget --show-progress -q https://www.dropbox.com/s/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth?dl=0 -O ./models/ColorizeArtistic_gen.pth
# Downloads "Stable" model (artistic = false)
!wget wget --show-progress -q https://www.dropbox.com/s/mwjep3vyqk5mkjc/ColorizeStable_gen.pth?dl=0 -O ./models/ColorizeStable_gen.pth

import fastai
from fasterai.visualize import *
from pathlib import Path

!touch /content/setup_finished

##◢ Colorize!!

In [0]:
#@title ## Colorize script {display-mode: "form"}
#@markdown After running this block, a file upload box will show. One or multiple jpg's or zip's can be submitted.

# Form
#@markdown ### Options

#@markdown Enable this to use the "Artistic" model (recommended). This model produces more vibrant colours.
#@markdown
#@markdown Disable to use the "Stable" model. This will often tend to do better on portraits, and natural landscapes.
#@markdown
#@markdown `Default: True`
artistic = True #@param {type:"boolean"}

#@markdown The render factor determines resolution at which the color portion of the image is rendered. Lower resolution will render faster, and colors also tend to look more vibrant.
#@markdown Older and lower quality images in particular will generally benefit by lowering the render factor.
#@markdown
#@markdown The default value of 35 has been carefully chosen and should work -ok- for most scenarios (but probably won't be the -best-).
#@markdown Higher render factors are often better for higher quality images, but the colors may get slightly washed out.
#@markdown
#@markdown `Default: 35`
render_factor = 35  #@param {type: "slider", min: 7, max: 45}

#@markdown Shows full sized image in the notebook (probably won't fit the screen).
#@markdown
#@markdown `Default: False`
show_fullsize = False #@param {type:"boolean"}

#@markdown Show a comparison of the original and colored image.
#@markdown
#@markdown `Default: False`
compare = False #@param {type:"boolean"}

#@markdown Generate images for every possible render factor. Recommended to use this with one image at a time.
#@markdown
#@markdown `Default: False`
iterate = False #@param {type:"boolean"}

#@markdown Enable to reuse the previously submitted files. This removes the file upload box.
#@markdown
#@markdown `Default: False`
keep_uploads = False #@param {type:"boolean"}

#@markdown Download all result images. A zip will be created for more multiple images.
#@markdown
#@markdown `Default: True`
download_result = True #@param {type:"boolean"}

%cd -q /content/DeOldify

import os
if not os.path.isfile("/content/setup_finished"):
    print("Setup did not complete! Run the Setup block above and try again.")
    
from google.colab import files
import shutil
import torch
import fastai
from fasterai.visualize import *
from pathlib import Path
from datetime import datetime
import warnings
import psutil
import gc

# Functions

def upload_files_dialog():
    %mkdir /content/uploaded /content/original_images /content/results &> /dev/null
    %cd -q /content/uploaded
    try:
        uploaded = files.upload()
    except TypeError:
        print("Browser session expired. Please rerun this block.")

    for filename in uploaded.keys():
        if filename[-4:] == ".zip":
            !unzip -df /content/original_images/ {filename}
        elif filename[-4:] == ".jpg":
            shutil.copy(filename, "/content/original_images/")
    !rm -rf /content/uploaded/*
    print("Uploaded", get_image_count(), "files.")
    %cd -q /content/DeOldify

def get_timestamp():
    timestamp = !date +%Y-%m-%d_%H-%M-%S
    return timestamp[0]

def get_image_paths():
    return Path("/content/original_images").glob("**/*.jpg")

def get_result_image_paths():
    return Path("/content/DeOldify/result_images").glob("**/*.jpg")

def get_image_count():
    files = list(get_image_paths())
    return len(list(files))

def zip_result():
    zipfile = "/content/results/DeOldify_" + get_timestamp() + ".zip"
    !zip -r {zipfile} result_images/*
    return zipfile

def clean_mem():
    torch.cuda.empty_cache()
    n = 2**14
    a_2GB = np.ones((n, n))  # RAM: +2GB
    del a_2GB  # RAM: -2GB
    gc.collect()

def colorize():
    pathlist = get_image_paths()
    for path in pathlist:
        original_path = str(path)
        if iterate:
            for i in range(10,46,2):
                image_path = colorizer.plot_transformed_image(original_path, render_factor=i, display_render_factor=True, figsize=(8,8))
                os.rename(str(image_path), str(image_path)[:-3] + '_' + str(render_factor) + '.jpg')
        else:
            image_path = colorizer.plot_transformed_image(original_path, render_factor=render_factor, compare=compare)  
        print(image_path)
        if show_fullsize:
            show_image_in_notebook(image_path)
        clean_mem()
        print("Memory in use:", psutil.virtual_memory().percent, "%")

# Actual script

warnings.simplefilter("ignore")

if not keep_uploads or get_image_count() == 0:
    upload_files_dialog()
else:
    print("keep_uploads is enabled, will use these files:")
    print(list(get_image_paths()))

colorizer = get_image_colorizer(artistic=artistic)

colorize()

if download_result and get_image_count() > 0:
    if get_image_count() == 1:
        for image in get_result_image_paths():
            files.download(str(image))
    elif get_image_count() > 1:
        zipfile = zip_result()
        files.download(zipfile)

if not keep_uploads:
    !rm -rf /content/original_images/*
!rm -rf /content/DeOldify/result_images/*

In [0]:
#@title Delete uploaded and generated files {display-mode: "form"}
#@markdown Useful if it crashes half way trough.
!rm -rf /content/uploaded/*
!rm -rf /content/original_images/*
!rm -rf /content/DeOldify/result_images/*